<a href="https://colab.research.google.com/github/Juako1314/Solar-2/blob/main/Calculadora%20solar%204.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:
%%capture
!pip install gradio
import gradio as gr
import math as mt
import matplotlib.pyplot as plt
import io
from PIL import Image
import numpy as np

In [66]:
def dtohms(fracday):
    hr = int(24 * fracday)
    min = int(((fracday * 24) - hr)  * 60)
    sec = int(((((fracday * 24) - hr)  * 60) - min) * 60)
    if min >= 10:
        if sec >=10:
            hms = "{}:{}:{}".format(hr, min, sec)
        else:
            hms = "{}:{}:0{}".format(hr, min, sec)
    else:
        if sec >= 10:
            hms = "{}:0{}:{}".format(hr, min, sec)
        else:
            hms = "{}:0{}:0{}".format(hr, min, sec)
    return hms

def mintohms(mins):
    hrs = int(mins / 60)
    minutes = int(mins - hrs * 60)
    secs = int(((mins - hrs * 60) - minutes)*60)
    if minutes >= 10:
        if secs >=10:
            hrminseg = "{}:{}:{}".format(hrs, minutes, secs)
        else:
            hrminseg  = "{}:{}:0{}".format(hrs, minutes, secs)
    else:
        if secs >= 10:
            hrminseg  = "{}:0{}:{}".format(hrs, minutes, secs)
        else:
            hrminseg  = "{}:0{}:0{}".format(hrs, minutes, secs)
    return hrminseg

def separa_dec(a):
  p1,p2,p3=a.find('°'),a.find('\''),a.find('\"')
  h,m,s=a[0:p1],a[int(p1+1):p2],a[int(p2+1):p3]
  return  float(h)+float(m)/60+float(s)/3600

def septime_dec(a):
  p1=a.find(':')
  p2=a[p1+1:].find(':')+p1+1
  h,m,s=a[0:p1],a[int(p1+1):p2],a[int(p2+1):]
  return  float(h)+float(m)/60+float(s)/3600




In [67]:
print(dtohms(0.5025596976231412))
print(mintohms(563.4681652046573))

12:03:41
9:23:28


In [68]:
#Calcula el dia juliano
def jd(fecha, hora_local, GMT):
    Date = []
    Meses,posicion,longyear = {"01" : 31, "02" : 28, "03" : 31, "04" : 30, "05" : 31, "06" : 30, "07" : 31, "08" : 31, "09" : 30, "10" : 31, "11" : 30, "12" : 31,},0,365
    for i in range(len(fecha)):
        Date.append(fecha[i])
    Año = int(Date[6] + Date[7] + Date[8] + Date[9])
    Mes = Date[3] + Date[4]
    Dia = int(Date[0]+Date[1])

    global f1900
    f1900 = 0
    for i in range(1900, Año):
        if i % 4 == 0 and (i % 100 != 0 or i % 400 == 0):
            f1900 += 366
        else:
            f1900 += 365

    if Año % 4 == 0 and (Año % 100 != 0 or Año % 400 == 0):
        Meses["02"],longyear = 29,366

    global meslength
    meslength=np.cumsum([0]+[i for i in Meses.values()])
    posicion=Dia+meslength[int(Mes)-1]

    julian = f1900 + posicion + 1 + 2415018.5 - GMT/24 + hora_local/24
    return julian

#Calcula el dia del año
def dia_anio(fecha):
    Date = []
    for i in range(len(fecha)):
        Date.append(fecha[i])
    Año = int(Date[6] + Date[7] + Date[8] + Date[9])
    Mes = Date[3] + Date[4]
    Dia = int(Date[0]+Date[1])

    Meses,posicion,longyear = {"01" : 31, "02" : 28, "03" : 31, "04" : 30, "05" : 31, "06" : 30, "07" : 31, "08" : 31, "09" : 30, "10" : 31, "11" : 30, "12" : 31,},0,365

    if Año % 4 == 0 and (Año % 100 != 0 or Año % 400 == 0):
      Meses["02"],longyear = 29,366

    meslength=np.cumsum([0]+[i for i in Meses.values()])
    posicion=Dia+meslength[int(Mes)-1]

    return posicion,longyear,meslength

#Calcula el siglo juliano
def jc(fecha, hora_local, GMT):
    century = (jd(fecha, hora_local, GMT) - 2451545) / 36525
    return century

#Calcula Geom Mean Long Sun en grados
def Geom_Mean_Long_Sun(fecha, hora_local, GMT):
    long_sun = 280.46646 + jc(fecha, hora_local, GMT) * (36000.76983 + jc(fecha, hora_local, GMT) * 0.0003032) % 360
    return long_sun

#Calcula Geom Mean Anom Sun en grados
def Geom_Mean_Anom_Sun(fecha, hora_local, GMT):
    anom_sun = 357.52911 + jc(fecha, hora_local, GMT) * (35999.05029 - 0.0001537 * jc(fecha, hora_local, GMT))
    return anom_sun

#Calcula la órbita terrestre excéntrica
def eccent(fecha, hora_local, GMT):
    orbit = 0.016708634 - jc(fecha, hora_local, GMT) * (0.000042037 + 0.0000001267 * jc(fecha, hora_local, GMT))
    return orbit

#calcula Sun Eq of Ctr
def Sun_Eq_Ctr(fecha, hora_local, GMT):
    eq_ctr = mt.sin(mt.radians(Geom_Mean_Anom_Sun(fecha, hora_local, GMT))) * (1.914602 - jc(fecha, hora_local, GMT) * (0.004817 + 0.000014 * jc(fecha, hora_local, GMT))) + mt.sin(mt.radians(2 * Geom_Mean_Anom_Sun(fecha, hora_local, GMT))) * (0.019993 - 0.000101 * jc(fecha, hora_local, GMT)) + mt.sin(mt.radians(3 * Geom_Mean_Anom_Sun(fecha, hora_local, GMT))) * 0.000289
    return eq_ctr

#Calcula Sun True Long en grados
def Sun_True_Long(fecha, hora_local, GMT):
    true_long = Geom_Mean_Long_Sun(fecha, hora_local, GMT) + Sun_Eq_Ctr(fecha, hora_local, GMT)
    return true_long

#Calcula Sun True Anom en grados
def Sun_True_Anom(fecha, hora_local, GMT):
    true_anom = Geom_Mean_Anom_Sun(fecha, hora_local, GMT) + Sun_Eq_Ctr(fecha, hora_local, GMT)
    return true_anom

#Calcula Sun Rad Vector en AUs
def Sun_Rad_Vector(fecha, hora_local, GMT):
    rad_vector = (1.000001018 * (1- eccent(fecha, hora_local, GMT) *eccent(fecha, hora_local, GMT))) / (1 + eccent(fecha, hora_local, GMT) * mt.cos(mt.radians(Sun_True_Anom(fecha, hora_local, GMT))))
    return rad_vector

#Calcula Sun App Long en grados
def Sun_App_Long(fecha, hora_local, GMT):
    app_long = Sun_True_Long(fecha, hora_local, GMT) - 0.00569 - 0.00478 * mt.sin(mt.radians(125.04 - 1934.136 * jc(fecha, hora_local, GMT)))
    return app_long

#Calcula Mean Obliq Ecliptic
def Mean_Obliq_Ecliptic(fecha, hora_local, GMT):
    obliq_ecliptic = 23 + (26 + ((21.448 - jc(fecha, hora_local, GMT) * (46.815 + jc(fecha, hora_local, GMT) * (0.00059 - jc(fecha, hora_local, GMT) * 0.001813)))) / 60) / 60
    return obliq_ecliptic

#Calcula Obliq Correction
def Obliq_Corr(fecha, hora_local, GMT):
    obliq_corr = Mean_Obliq_Ecliptic(fecha, hora_local, GMT) + 0.00256 * mt.cos(mt.radians(125.04 - 1934.136 * jc(fecha, hora_local, GMT)))
    return obliq_corr

#Calcula Sun RT Ascen
def Sun_Rt_Ascen(fecha, hora_local, GMT):
    Ascen = mt.degrees(mt.atan2(mt.cos(mt.radians(Obliq_Corr(fecha, hora_local, GMT))) * mt.sin(mt.radians(Sun_App_Long(fecha, hora_local, GMT))) , mt.cos(mt.radians(Sun_App_Long(fecha, hora_local, GMT)))))
    return Ascen

#Calcula la declinacion solar
def Sun_Declination(fecha, hora_local, GMT):
    declination = mt.degrees(mt.asin(mt.sin(mt.radians(Obliq_Corr(fecha, hora_local, GMT))) * mt.sin(mt.radians(Sun_App_Long(fecha, hora_local, GMT)))))
    return declination

#Calcula var y
def var_y(fecha, hora_local, GMT):
    var = mt.tan(mt.radians(Obliq_Corr(fecha, hora_local, GMT) / 2)) * mt.tan(mt.radians(Obliq_Corr(fecha, hora_local, GMT) / 2))
    return var

#Calcula la ecuacion del tiempo
def Eq_Time(fecha, hora_local, GMT):
    global of_time
    of_time = 4 * mt.degrees(var_y(fecha, hora_local, GMT) * mt.sin(2 * mt.radians(Geom_Mean_Long_Sun(fecha, hora_local, GMT))) - 2 * eccent(fecha, hora_local, GMT) * mt.sin(mt.radians(Geom_Mean_Anom_Sun(fecha, hora_local, GMT))) + 4 * eccent(fecha, hora_local, GMT) * var_y(fecha, hora_local, GMT) * mt.sin(mt.radians(Geom_Mean_Anom_Sun(fecha, hora_local, GMT))) * mt.cos(2 * mt.radians(Geom_Mean_Long_Sun(fecha, hora_local, GMT))) - 0.5 * var_y(fecha, hora_local, GMT) * var_y(fecha, hora_local, GMT) * mt.sin(4 * mt.radians(Geom_Mean_Long_Sun(fecha, hora_local, GMT))) - 1.25 * eccent(fecha, hora_local, GMT) * eccent(fecha, hora_local, GMT) * mt.sin(2 * mt.radians(Geom_Mean_Anom_Sun(fecha, hora_local, GMT))))
    minut = int(of_time)
    seconds = int((of_time - minut) * 60)
    if abs(seconds) >= 10:
        time = "{}:{}".format(minut, abs(seconds))
    else:
        time = "{}:0{}".format(minut, abs(seconds))
    return time

#Calcula HA Sunrise
def Ha_Sunrise(fecha, hora_local, GMT,Latitud):
    ha = mt.degrees(mt.acos(mt.cos(mt.radians(90.833)) / (mt.cos(mt.radians(Latitud)) * mt.cos(mt.radians(Sun_Declination(fecha, hora_local, GMT)))) - mt.tan(mt.radians(Latitud)) * mt.tan(mt.radians(Sun_Declination(fecha, hora_local, GMT)))))
    return ha

#Calcula el mediodia solar
def Solar_Noon(fecha, hora_local, GMT, Longitud):
    global noon
    noon = (720 - 4 * Longitud - of_time+ GMT * 60) / 1440
    return dtohms(noon)

#Calcula el amanecer
def Sunrise(fecha, hora_local, GMT, Latitud):
    rise = (noon * 1440 - Ha_Sunrise(fecha, hora_local, GMT, Latitud) * 4)/1440
    return dtohms(rise)

#Calcula el atardecer
def Sunset(fecha, hora_local, GMT, Latitud):
    global noon
    sset = (noon * 1440 + Ha_Sunrise(fecha, hora_local, GMT, Latitud) * 4) / 1440
    return dtohms(sset)

#Duracion de la luz solar
def Sunlight(fecha, hora_local, GMT, Latitud):
    light = 8 * Ha_Sunrise(fecha, hora_local, GMT, Latitud)
    return mintohms(light)

#Calcula el tiempo solar
def True_Solar_Time(hora_local, Longitud, GMT):
    global Solar_Time
    Solar_Time = ((hora_local * 60 + of_time + 4 * Longitud - 60 * GMT) % 1440)
    print(Solar_Time)
    return mintohms(Solar_Time)

#Calcula la hora angular
def Hour_Angle():
    if Solar_Time / 4 < 0:
        Ahour = (Solar_Time / 4) + 180
    else:
        Ahour = (Solar_Time / 4) - 180
    return Ahour

#Calcula el angulo cenital
def Zenith_Angle(fecha, hora_local, GMT, Latitud):
    zenith = mt.degrees(mt.acos(mt.sin(mt.radians(Latitud)) * mt.sin(mt.radians(Sun_Declination(fecha, hora_local, GMT))) + mt.cos(mt.radians(Latitud)) * mt.cos(mt.radians(Sun_Declination(fecha, hora_local, GMT))) * mt.cos(mt.radians(Hour_Angle()))))
    return zenith

#Calcula la elevacion del sol
def Solar_Elevation(fecha, hora_local, GMT, Latitud):
    elevation = 90 - Zenith_Angle(fecha, hora_local, GMT, Latitud)
    return elevation

#Calcula la refraccion atmosferica
def Atm_Refraction(fecha, hora_local, GMT, Latitud):
    if Solar_Elevation(fecha, hora_local, GMT, Latitud) > 85:
        refraction = 0 / 3600
    elif Solar_Elevation(fecha, hora_local, GMT, Latitud) > 5:
        refraction = (58.1 / mt.tan(mt.radians(Solar_Elevation(fecha, hora_local, GMT, Latitud))) - 0.07/(mt.tan(mt.radians(Solar_Elevation(fecha, hora_local, GMT, Latitud)))) ** 3 + 0.000086/(mt.tan(mt.radians(Solar_Elevation(fecha, hora_local, GMT, Latitud)))) ** 5) / 3600
    elif Solar_Elevation(fecha, hora_local, GMT, Latitud) > -0.575:
        refraction = (1735 + Solar_Elevation(fecha, hora_local, GMT, Latitud) * (-518.2 + Solar_Elevation(fecha, hora_local, GMT, Latitud) * (103.4 + Solar_Elevation(fecha, hora_local, GMT, Latitud) * (-12.79 + Solar_Elevation(fecha, hora_local, GMT, Latitud) * 0.711)))) / 3600
    else:
        refraction = (-20.772 / mt.tan(mt.radians(Solar_Elevation(fecha, hora_local, GMT, Latitud))) / 3600)
    return refraction

#Calcula la elevacion solar corregida
def Solar_Elevation_Corr(fecha, hora_local, GMT, Latitud):
    corr_elevation = Atm_Refraction(fecha, hora_local, GMT, Latitud) + Solar_Elevation(fecha, hora_local, GMT, Latitud)
    return corr_elevation

#Calcula el angulo acimutal
def Azimuth_Angle(fecha, hora_local, GMT, Latitud):
    if Hour_Angle() > 0:
        azimuth = (mt.degrees(mt.acos(((mt.sin(mt.radians(Latitud)) * mt.cos(mt.radians(Zenith_Angle(fecha, hora_local, GMT, Latitud)))) - mt.sin(mt.radians(Sun_Declination(fecha, hora_local, GMT)))) / (mt.cos(mt.radians(Latitud)) * mt.sin(mt.radians(Zenith_Angle(fecha, hora_local, GMT, Latitud)))))) + 180) % 360
    else:
        azimuth = (540 - mt.degrees(mt.acos(((mt.sin(mt.radians(Latitud)) * mt.cos(mt.radians(Zenith_Angle(fecha, hora_local, GMT, Latitud)))) - mt.sin(mt.radians(Sun_Declination(fecha, hora_local, GMT))))/(mt.cos(mt.radians(Latitud)) * mt.sin(mt.radians(Zenith_Angle(fecha, hora_local, GMT, Latitud))))))) % 360
    return azimuth


In [69]:
acumulados = dia_anio("31/12/2024")[2]
longyear = 366

In [70]:
def geometria_solar(fecha, hora_local_str, GMT, Longit,  latit, es_oes, Nor_sur):


    Longitud = float(separa_dec(Longit))
    Latitud = float(separa_dec(latit))
    #Dia, longyear,acumulados=dia_anio(fecha)
    hora_local = septime_dec(hora_local_str)
    if Nor_sur == False:Latitud*=-1
    if es_oes == False:Longitud*=-1

    ########### Parámetros calculados ################
    Dia_juliano = round(jd(fecha, hora_local, GMT), 4)
    Siglo_juliano = round((jd(fecha, hora_local, GMT) - 2451545) / 36525, 6)
    Mean_long = round(Geom_Mean_Long_Sun(fecha, hora_local, GMT) , 4)
    Anom_long = round(Geom_Mean_Anom_Sun(fecha, hora_local, GMT) , 4)
    Orbita_excentrica = round(eccent(fecha, hora_local, GMT), 10)
    Eq_de_ctr = round(Sun_Eq_Ctr(fecha, hora_local, GMT), 4)
    True_long = round(Sun_True_Long(fecha, hora_local, GMT), 4)
    True_anom = round(Sun_True_Anom(fecha, hora_local, GMT), 4)
    vector = round(Sun_Rad_Vector(fecha, hora_local, GMT), 5)
    App_long = round(Sun_App_Long(fecha, hora_local, GMT), 4)
    obliq_ecliptic = round(Mean_Obliq_Ecliptic(fecha, hora_local, GMT), 8)
    corr_obliq = round(Obliq_Corr(fecha, hora_local, GMT), 4)
    rt_ascen = round(Sun_Rt_Ascen(fecha, hora_local, GMT), 4)
    declinacion = round(Sun_Declination(fecha, hora_local, GMT), 4)
    vary = round(var_y(fecha, hora_local, GMT), 8)
    eqtime = Eq_Time(fecha, hora_local, GMT)
    ha = round(Ha_Sunrise(fecha, hora_local, GMT,Latitud))
    mediodia = Solar_Noon(fecha, hora_local, GMT, Longitud)
    amanecer = Sunrise(fecha, hora_local, GMT, Latitud)
    atardecer = Sunset(fecha, hora_local, GMT, Latitud)
    luz_solar = Sunlight(fecha, hora_local, GMT, Latitud)
    tiempo_solar = True_Solar_Time(hora_local, Longitud, GMT)
    hora_angular = round(Hour_Angle(), 4)
    cenital = round(Zenith_Angle(fecha, hora_local, GMT, Latitud), 4)
    elevacion_solar = round(Solar_Elevation(fecha, hora_local, GMT, Latitud), 4)
    refraccion = Atm_Refraction(fecha, hora_local, GMT, Latitud)
    corr_elevacion = round(Solar_Elevation_Corr(fecha, hora_local, GMT, Latitud), 4)
    acimutal = round(Azimuth_Angle(fecha, hora_local, GMT, Latitud), 4)


    angles, x = [], 0
    print(fecha)

    #for i in range(1, 361):

    #    dsg = Sun_Declination() *(180/mt.pi)
    #    x += dsg
    #    if i % 30 == 0:
    #        prom = x / 30
    #        angles.append(prom)
    #        x = 0
    #    else:
    #        continue

    #for i in range(12):
    #    angles[i] = Latitud - angles[i]


    return declinacion, eqtime, tiempo_solar, hora_angular, cenital, acimutal, amanecer, atardecer, mediodia, luz_solar














    #Calcula los angulos de inclinacion tomando en cuenta solamente la radiacion directa

    #angles,x = [],0

    #angles,x,maac = [],0,0

    #for i in range(12):
      #for j in (range(acumulados[i]+1,acumulados[i+1]+1)):
        #ds=Latitud-decsol( fracyear(j,12,longyear) ) *(180/mt.pi)
        #mair=np.cos(ds*mt.pi/180)/(1-0.01672*mt.cos(0.9856*(j-4)))**2
        #maac+=mair
        #x +=ds*mair
        #prom=x/maac
      #x,maac=0,0
      #angles.append(prom)

    #for i in range(1, 361):
     #   dsg = decsol( fracyear(i,12,longyear) ) *(180/mt.pi)
      #  x += dsg
       # if i % 30 == 0:
        #    prom = x / 30
         #   angles.append(prom)
          #  x = 0
       # else:
        #    continue

#    for i in range(12):
#        angles[i] = Latitud - angles[i]

    #Month = ['Jan', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Agos', 'Sep', 'Oct', 'Nov', 'Dec']


    # Plotting Line Graph
    #plt.title("Ángulos de inclinación óptimos mensuales")
    #plt.xlabel('Meses')
    #plt.ylabel("Ángulo óptimo")
    #plt.bar(Month, angles)

    # Getting the current figure and save it in the variable.
    #fig = plt.gcf()

    # Create a Function for Converting a figure to a PIL Image.

    #buf = io.BytesIO()
    #fig.savefig(buf)
    #buf.seek(0)
    #imge = Image.open(buf)

    #return round(año_fraccional,4), round(Declinacion_solar_grados,4), eqt, tst, round(Hora_angular_grados, 4), round(Angulo_cenital_grados, 4), round(Angulo_acimutal_grados,4), Amt, Att, Met, imge
# geometria_solar("10/10/2023", "22:34:00", -7, "108°59'03\"","25°45'03\"", True, True) Prueba de la función


In [97]:
global z
z = '"'
with gr.Blocks() as demo:

##################################################################
    from datetime import datetime
    from datetime import date
    import pytz

    hora = datetime.now(pytz.timezone('America/Mazatlan'))
    hmoch=hora.strftime("%H:%M:%S")
    dmoch=hora.strftime("%d/%m/%Y")
    zonas=[]
    for tz in pytz.all_timezones:
      zonas.append(tz)


##################################################################
    with gr.Row():
        with gr.Column():
            gr.Markdown(
                """
                # Calculadora solar Tecnológico Nacional de México / Instituto Tecnológico de Los Mochis
                Elaborada en la División de Estudios de Posgrado e Investigación del ITLM.\n
                Por: **Joaquín Montes Vega**\n
                Supervisado por: **Dr. Alexander Mendoza Acosta**
                """)
        with gr.Column():
            with gr.Row():
                gr.Image(value = "https://media.licdn.com/dms/image/C4E0BAQHjH7hPbCYYqw/company-logo_200_200/0/1582078609694?e=2147483647&v=beta&t=8MUC0l51wSsbn6JqIInY3bel6prnE08Gfujk31oD7Iw")
                gr.Image(value = "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRNKN_P93RL4NPAMAVHK_tROTKraAbzQjQbzybBgGh3XS4qYCME7vaOpbFg0c_xJ_fdn2s&usqp=CAU")

        #Entradas
    with gr.Tab("Datos"):
        with gr.Row():
            zona=gr.Dropdown(label="Zona horaria",choices=zonas,value='America/Mazatlan', interactive = True)
            Day = gr.Textbox(label="Fecha", placeholder = "dd/mm/yyyy", info = "Ej. 01/05/2023",value=dmoch)
            local_hour = gr.Textbox(label = "Hora local", placeholder = "hh:mm:ss", info = "Ej. 17:04:09",value=hmoch)
            GMTs = gr.Number(label = "Zona horaria", info = "Ej. si es -07:00, solo se ingresara el -7",value=-7)
        with gr.Row():
            Longitude = gr.Textbox(label = "Longitud", placeholder = "ggg°mm'ss{}".format(z), info = "Ej. 5°10'05{}".format(z),value="108°59'03\"")
            Latitude = gr.Textbox(label = "Latitud", placeholder = "gg°mm'ss{}".format(z), info = "Ej. 20°04'46{}".format(z),value="25°45'03\"")
        with gr.Row():
            es_oes = gr.Checkbox(label = "Este", value = False, interactive = True)
            Nor_sur = gr.Checkbox(label = "Norte", value = True, interactive = True)
        with gr.Row():
            gr.Markdown(
                """
                Las ecuaciones empleadas fueron tomadas de la National Oceanic and Atmospheric Administration
                [(NOAA)](https://gml.noaa.gov/grad/solcalc/)
                """
            )


    #Salidas
    with gr.Tab("Resultados"):

        with gr.Row():
            Declinacion = gr.Number(label = "Declinacion solar (°)")
            eqt_min_seg = gr.Textbox(label = "Ecuacion del tiempo (min)")
            Solartime = gr.Textbox(label = "Tiempo Solar (hh:mm:ss)")
            angular_hour = gr.Number(label = "Hora angular (°)")
            zenith = gr.Number(label = "Angulo cenital (°)")
        with gr.Row():
            azimuth = gr.Number(label = "Angulo acimutal (°)")
            sunrise = gr.Textbox(label = "Amanecer (hh:mm:ss)")
            sunset = gr.Textbox(label = "Atardecer (hh:mm:ss)")
            hday = gr.Textbox(label = "Mediodia Solar (hh:mm:ss)")
            Sunl = gr.Textbox(label = "Duracion de la luz del sol")

        #with gr.Row():
        #    angles = gr.Image(label = "Ángulos de inclinacion óptimos", info = "Sólo se toma en cuenta la radiacion directa")
        with gr.Row():
            gr.Markdown(
                """
                Las ecuaciones empleadas fueron tomadas de la National Oceanic and Atmospheric Administration
                [(NOAA)](https://gml.noaa.gov/grad/solcalc/)
                """
            )



    zona.change(fn=lambda x: (datetime.now(pytz.timezone(x)).strftime("%H:%M:%S"),datetime.now(pytz.timezone(x)).strftime("%d/%m/%Y"),float(datetime.now(pytz.timezone(x)).strftime("%H"))-float(datetime.now(pytz.timezone('UTC')).strftime("%H"))), inputs=zona, outputs=[local_hour,Day,GMTs])
    es_oes.change(fn=lambda x: abs(float(x)) if es_oes.value==True else -(x),inputs=GMTs, outputs=GMTs )


    Calcular_btn = gr.Button("Calcular")
    Calcular_btn.click(fn=geometria_solar,
                       inputs=[Day, local_hour, GMTs, Longitude,  Latitude, es_oes, Nor_sur],
                       outputs=[Declinacion, eqt_min_seg, Solartime, angular_hour, zenith, azimuth, sunrise, sunset, hday, Sunl]#, #angles]
                       )

demo.launch(share = True)


Thanks for being a Gradio user! If you have questions or feedback, please join our Discord server and chat with us: https://discord.gg/feTf9x3ZSB
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://4af26e8bd2fde52513.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
